In [5]:
from module.bis_selenium import BisSelenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import polars as pl
from bs4 import BeautifulSoup

In [11]:
import time
import os
from dotenv import load_dotenv; load_dotenv()

driver_path = r'C:\Users\341137\src\driver\chromedriver.exe'
auth_id = os.environ.get('AUTH_ID')
auth_pass = os.environ.get('AUTH_PASS')

bis_selenium = BisSelenium(driver_path)
bis_selenium.login(auth_id, auth_pass)
bis_selenium.move_to_timesales()

# element_selecter
pattern_selecter = bis_selenium.find_element(By.CSS_SELECTOR, '#displayPatternSelect1')
date_selecter = bis_selenium.find_element(By.CSS_SELECTOR, '#date')
hour_selecter = bis_selenium.find_element(By.CSS_SELECTOR, '#hour')  
report_viewer = bis_selenium.find_element(By.CSS_SELECTOR, '#viewJikanichiran')

# 条件選択部分
bis_selenium.find_elements(By.CSS_SELECTOR, '#kaishaCodeRadio')[2].click()
Select(pattern_selecter).select_by_value('7')

# 組織選択
org_list = ['0003_12000', '0004_13000']
bis_selenium.select_orgs(*org_list)

# 商品分類選択
product_list = ['[211]グロサリー', '[214]リカー', '[221]デイリーフーズ']
bis_selenium.select_products(*product_list)

# 日付選択
bis_selenium.input_without_JS(date_selecter, '2023/07/30')
Select(hour_selecter).select_by_value('10')

# 画面遷移
report_viewer.click()
bis_selenium.move_to_new_window()
first_frame = bis_selenium.find_element(By.CSS_SELECTOR, '#Dottom')
bis_selenium.switch_to.frame(first_frame)

# first_fetch
time_selector = bis_selenium.find_element(By.CSS_SELECTOR, '#TIME_SELECT')
bottom_frame = bis_selenium.find_element(By.CSS_SELECTOR, '#bottomFrame')
bis_selenium.switch_to.frame(bottom_frame)
html = bis_selenium.page_source.encode('utf-8')

bis_selenium.quit()

0003_12000
0004_13000
switch_window:時間帯別売上速報(一覧)


In [19]:
tr_list = tbody_html.find_all('tr')
columns = []
for i in range(len(tr_list[0].find_all('td'))):
    columns.append(tr_list[0].find_all('td')[i].get_text())

# table = pd.DataFrame()
# table['columns'] = columns

row_list = []
for j in range(1, len(tr_list)):
    row_data = []
    for k in range(len(tr_list[j].find_all('td'))):
        row_data.append(tr_list[j].find_all('td')[k].get_text().replace('\n', ''))
    # append_row = pd.DataFrame(row_data)
    row_list.append(row_data)

table = pd.DataFrame(row_list, columns = columns)
table.columns = table.columns.str.replace('\n', '').str.strip()

display(table)

,店舗CD,店舗,グループCD,グループ,直荒高,直荒率,直荒 予算率,直荒 昨年率,値入率,必要 値入率,...,売上 予算比,売上 昨対比 (全規模),売変率,売変 予算率,売変 昨年率,月末在庫高,月末在庫予算比,月末在庫昨対比 (全規模),交差主義比率,GMROI
0,0000002160,イオン亀田,,イオン亀田計,"16,683",23.25,23.41,22.77,26.73,27.35,...,98.34,97.60,5.58,5.89,6.69,"28,414",106.09,106.92,56.95,9.67
1,,,112,子供,0,0.00,,,0.00,,...,,,0.00,,,0,,,0.00,0.00
2,,,113,ベビー,1,6.01,10.71,10.65,13.81,19.10,...,43.44,44.18,8.91,10.71,7.99,48,59.61,58.43,1.37,0.19
3,,,116,雑貨,56,56.09,0.00,0.00,51.28,100.00,...,0.00,0.00,0.68,0.00,0.00,258,85.84,"6,706.25",20.87,5.21
4,,,122,インナーカジュアル,8,52.99,,,0.00,,...,,,5.42,,,451,,,1.75,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6186,,,381,ビューティ,2,38.25,36.36,38.11,35.59,30.38,...,40.66,41.08,0.54,0.00,0.49,84,123.52,98.02,2.60,0.50
6187,,,382,ファーマシー,21,36.62,38.00,33.51,44.43,44.98,...,112.90,123.36,7.00,2.00,2.30,295,256.56,209.95,7.27,1.50
6188,,,384,デイリーコンビニエンス,114,21.65,25.54,24.04,29.62,31.81,...,80.78,55.41,2.63,6.46,0.86,843,116.96,126.49,14.29,2.49
6189,,,386,ガーデニング・グリナリー,191,28.00,20.87,20.34,35.29,36.11,...,118.42,119.46,14.66,19.13,21.37,179,198.74,189.94,139.00,27.71


In [20]:
table.columns

Index(['店舗CD', '店舗', 'グループCD', 'グループ', '直荒高', '直荒率', '直荒   予算率', '直荒   昨年率',
       '値入率', '必要   値入率', '昨年   値入率', '売上高', '売上   予算比', '売上     昨対比  (全規模)',
       '売変率', '売変   予算率', '売変   昨年率', '月末在庫高', '月末在庫予算比', '月末在庫昨対比   (全規模)',
       '交差主義比率', 'GMROI'],
      dtype='object')

In [19]:
tr_list = tbody_html.find_all('tr')
print(len(tr_list))
print(tr_list[0])

6192
<tr>
<td class="x2" nowrap="" style="vertical-align:bottom">
<br/>店舗<br/>CD    </td>
<td class="x2" nowrap="" style="vertical-align:bottom">
<br/> <br/>店舗</td>
<td class="x2" nowrap="" style="vertical-align:bottom">
<br/>グループ<br/>CD          </td>
<td class="x2" nowrap="" style="vertical-align:bottom">
<br/> <br/>グループ</td>
<td class="x3" nowrap="" style="vertical-align:bottom">
<br/> <br/>直荒高</td>
<td class="x4" nowrap="" style="vertical-align:bottom">
<br/> <br/>直荒率</td>
<td class="x5" nowrap="" style="vertical-align:bottom">
<br/>直荒   <br/>予算率</td>
<td class="x6" nowrap="" style="vertical-align:bottom">
<br/>直荒   <br/>昨年率</td>
<td class="x7" nowrap="" style="vertical-align:bottom">
<br/> <br/>値入率</td>
<td class="x8" nowrap="" style="vertical-align:bottom">
<br/>必要   <br/>値入率</td>
<td class="x9" nowrap="" style="vertical-align:bottom">
<br/>昨年   <br/>値入率</td>
<td class="x10" nowrap="" style="vertical-align:bottom">
<br/> <br/>売上高</td>
<td class="x11" nowrap="" style="vertical-ali

22

In [2]:
length = []
for i in range(len(tr_list)):
    length.append(len(tr_list[i].find_all('td')))
    
pd.Series(length).unique()

NameError: name 'tr_list' is not defined

In [1]:
import bs4